In [148]:
# import libraries
import pandas as pd
pd.set_option('display.max_colwidth', None) #setting max colwidth to view the entire dataset when using the print() command
import matplotlib.pyplot as plt
import numpy as np


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier



from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from IPython.display import display

# import the files
data = open("./dataset/breast-cancer.data")
feat = open("./dataset/breast-cancer.names")

data = data.read()
feat = feat.read()

# replace missing dataset attributes to NAN
data = data.replace('?','')

from io import StringIO

# convert data from str to dataframe
data = StringIO(data)
data = pd.read_csv(data, sep=",")

data.columns = ['class', 'age', 'menopause', 'tumour_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irrad']

data['class'] = data['class'].replace(['no-recurrence-events','recurrence-events'], [0,1])
data['age'] = data['age'].replace(['20-29', '30-39','40-49','50-59','60-69','70-79'],[0,1,2,3,4,5])
data['menopause'] = data['menopause'].replace(['premeno','ge40','lt40'],[0,1,2])
data['tumour_size'] = data['tumour_size'].replace(['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54'],[0,1,2,3,4,5,6,7,8,9,10])
data['inv_nodes'] = data['inv_nodes'].replace(['0-2','3-5','6-8','9-11','12-14','15-17','24-26'],[0,1,2,3,4,5,6])
data['node_caps'] = data['node_caps'].replace(['no','yes'],[0,1])
data['breast'] = data['breast'].replace(['left','right'],[0,1])
data['breast_quad'] = data['breast_quad'].replace(['left_low','left_up','right_up','right_low','central'],[0,1,2,3,4])
data['irrad'] = data['irrad'].replace(['no','yes'],[0,1])

# replace missing data 
data['node_caps'] = data['node_caps'].fillna(5)
data['breast_quad'] = data['breast_quad'].fillna(8)

X = data.drop(['class'], axis = 1)
y = data['class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

X_train['size_agg'] = X_train['tumour_size'] + X_train['deg_malig'] 

X_test['size_agg'] = X_test['tumour_size'] + X_test['deg_malig']

In [149]:
"""
def model(X_train, X_test, y_train, y_test, feat, feat2, feat3):

    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
    model = DecisionTreeClassifier() 

    model.fit(X_resampled, y_resampled)

    y_score = model.predict_proba(X_test)[:, 1]

    y_pred = model.predict(X_test)

    acc = model.score(X_train, y_train)
    f1 = f1_score(y_test, y_pred, average='binary')
    roc = roc_auc_score(y_test, y_score)
    rec = recall_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred)

    if f1 > 0.68:
        print('\n','Analysed feature:',feat,'//', feat2,'//', feat3)
        print(X_train.columns)
        print(ros)
        print('Acc:',acc)
        print('F1:',f1)
        print('ROC-AUC:',roc)
        print('Recc:',rec)
        print('Prec:',pre)  
   

"""

"\ndef model(X_train, X_test, y_train, y_test, feat, feat2, feat3):\n\n    ros = RandomOverSampler(random_state=42)\n    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)\n    model = DecisionTreeClassifier() \n\n    model.fit(X_resampled, y_resampled)\n\n    y_score = model.predict_proba(X_test)[:, 1]\n\n    y_pred = model.predict(X_test)\n\n    acc = model.score(X_train, y_train)\n    f1 = f1_score(y_test, y_pred, average='binary')\n    roc = roc_auc_score(y_test, y_score)\n    rec = recall_score(y_test, y_pred)\n    pre = precision_score(y_test, y_pred)\n\n    if f1 > 0.68:\n        print('\n','Analysed feature:',feat,'//', feat2,'//', feat3)\n        print(X_train.columns)\n        print(ros)\n        print('Acc:',acc)\n        print('F1:',f1)\n        print('ROC-AUC:',roc)\n        print('Recc:',rec)\n        print('Prec:',pre)  \n   \n\n"

In [150]:
"""
features = ['age', 'menopause', 'tumour_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irrad']
def mix(features, X_train, X_test, y_train, y_test):
    for feat in features:
        for feat2 in features:
            for feat3 in features:
                important = ['tumour_size', 'breast_quad', 'size_agg']
                important.append(feat)
                important.append(feat2)
                important.append(feat3)
                X_train_n = X_train[important]
                X_test_n = X_test[important]
                model(X_train_n, X_test_n, y_train, y_test, feat, feat2, feat3)

"""

"\nfeatures = ['age', 'menopause', 'tumour_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irrad']\ndef mix(features, X_train, X_test, y_train, y_test):\n    for feat in features:\n        for feat2 in features:\n            for feat3 in features:\n                important = ['tumour_size', 'breast_quad', 'size_agg']\n                important.append(feat)\n                important.append(feat2)\n                important.append(feat3)\n                X_train_n = X_train[important]\n                X_test_n = X_test[important]\n                model(X_train_n, X_test_n, y_train, y_test, feat, feat2, feat3)\n\n"

In [151]:
#mix(features, X_train, X_test, y_train, y_test)

In [152]:
important_features = ['tumour_size', 'breast_quad', 'size_agg']
X_train = X_train[important_features]
X_test = X_test[important_features]

In [153]:
def mix(X_train, y_train):
    k = 1
    while k < 400:
        print(k)
        ros = RandomOverSampler(random_state=42)
        X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
        model = RandomForestClassifier(max_samples=k, criterion='log_loss')
        model.fit(X_resampled, y_resampled)
        y_score = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)
        acc = model.score(X_train, y_train)
        f1 = f1_score(y_test, y_pred, average='binary')
        roc = roc_auc_score(y_test, y_score)
        rec = recall_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)        
        if f1 >= 0.69:
            print('YES, with:', k)
            print('Acc:',acc)
            print('F1:',f1)
            print('ROC-AUC:',roc)
            print('Recc:',rec)
            print('Prec:',pre)  
        k += 1




In [154]:
mix(X_train, y_train)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


ValueError: `max_samples` must be in range 1 to 320 but got value 321

class sklearn.ensemble.RandomForestClassifier(**n_estimators=100**, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html